# Oanda Error Debuging

In [3]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta, timezone # Need to import timezone seperatly
import time
import v20.errors
import traceback

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        # Ensure bar_length is a Timedelta
        if isinstance(bar_length, str):
             self.bar_length = pd.to_timedelta(bar_length)
        else:
             self.bar_length = bar_length # Assume it's already a Timedelta if not string

        self.tick_data = pd.DataFrame()
        self.raw_data = None # Will store historical data (index might be aware initially)
        self.data = None      # Will store data with strategy signals (index likely naive)
        self.last_bar = None  # Stores the timestamp of the last full bar (NAIVE UTC)
        self.units = units
        self.position = 0
        self.profits = []

        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************

    def get_most_recent(self, days = 5):
        print("\n--- Entering get_most_recent ---") # DEBUG
        loop_counter = 0 # DEBUG counter
        max_loops = 10 # DEBUG limit - Increase if needed, but fix clock first!

        while True:
            loop_counter += 1 # DEBUG
            print(f"\nLoop Iteration: {loop_counter}") # DEBUG

            if loop_counter > max_loops: # DEBUG safety break
                print(f"DEBUG: Exiting loop after {max_loops} iterations. Check API errors or clock.")
                # Crucial: Raise an error or handle the failure state properly
                # If this fails, the bot cannot initialize properly.
                raise SystemExit(f"Failed to get recent data after {max_loops} attempts. Check system clock and OANDA connection.")
                # break # Avoid breaking without setting data if possible

            time.sleep(2) # Prevent rapid-fire requests on error

            # Use aware UTC timestamps for API requests
            # *** VERIFY YOUR SYSTEM CLOCK IS CORRECT ***
            now_aware = datetime.now(timezone.utc)
            past_aware = now_aware - timedelta(days = days)

            # Format for printing/logging clarity
            now_str = now_aware.strftime('%Y-%m-%d %H:%M:%S %Z')
            past_str = past_aware.strftime('%Y-%m-%d %H:%M:%S %Z')
            print(f"Fetching history from {past_str} to {now_str}") # DEBUG API call times

            try:
                # Request data - localize=False is default and usually returns UTC anyway
                df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
                                      granularity = "S5", price = "M", localize = False)

                # Check if the response was successful (tpqoa might handle some errors internally,
                # but the ResponseNoField error shows it doesn't catch everything)
                # A more robust check might involve inspecting the response before accessing 'c'
                if not isinstance(df, pd.DataFrame) or 'c' not in df.columns:
                     print(f"Warning: get_history did not return expected DataFrame format. Response: {df}")
                     # Consider adding more specific error handling based on tpqoa's return types on error
                     continue

                df = df.c.dropna().to_frame() # Select close price, drop NaNs

                if df.empty:
                    print("Warning: get_history returned DataFrame with no valid close prices. Waiting and retrying.") # DEBUG
                    continue # Skip rest of loop and try again

                df.rename(columns = {"c":self.instrument}, inplace = True)
                print(f"  History data shape BEFORE resample: {df.shape}")
                if not df.empty:
                    print(f"  History data index TZ before resample: {df.index.tz}") # DEBUG Check TZ (Should be UTC)
                    # print(f"  History data tail BEFORE resample:\n{df.tail(3)}") # Optional Debug

                # Resample to desired bar length. label='right' aligns the timestamp to the end of the bar.
                # The index should remain timezone-aware (UTC) after resampling.
                # Drop the last bar? -> Reconsider this. The loop condition handles waiting.
                # Let's keep the last bar for now unless there's a strong reason not to.
                # df_resampled = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1] <--- Original
                df_resampled = df.resample(self.bar_length, label = "right").last().dropna() # Keep all bars

                print(f"  Resampled data shape (after dropna): {df_resampled.shape}")
                if df_resampled.empty:
                    print("Warning: Resampled DataFrame is empty after dropna. Waiting and retrying.")
                    continue
                print(f"  Resampled data index TZ after resample: {df_resampled.index.tz}") # DEBUG Check TZ (Should be UTC)
                # print(f"  Resampled data tail:\n{df_resampled.tail(3)}") # Optional Debug

                # Store the raw resampled data (index is aware UTC)
                self.raw_data = df_resampled.copy()

                # --- Set last_bar to NAIVE UTC ---
                # This is a key decision: Internal logic will use naive UTC for comparisons.
                if not self.raw_data.empty:
                    aware_last_bar = self.raw_data.index[-1] # This is TZ-AWARE UTC
                    self.last_bar = aware_last_bar.tz_localize(None) # Convert to NAIVE UTC
                    print(f"DEBUG: Initial self.last_bar set to NAIVE UTC: {self.last_bar}")
                else:
                    print("WARNING: raw_data is empty after resample/dropna, cannot set last_bar. Retrying.")
                    continue

                # --- Check if the last bar is recent enough ---
                # Use consistent NAIVE UTC times for the loop's exit check
                current_check_time_naive = datetime.now(timezone.utc).replace(tzinfo=None)
                if self.last_bar is None: # Should not happen if code above worked
                    print("WARNING: self.last_bar is None before time diff check. Retrying.")
                    continue

                # Perform comparison with naive timestamps
                time_diff = current_check_time_naive - self.last_bar

                print(f"  Current Naive UTC Time:    {current_check_time_naive}")
                print(f"  Last Bar Time (Naive UTC): {self.last_bar}")
                print(f"  Bar Length:                {self.bar_length}")
                print(f"  Time Difference:           {time_diff}")
                print(f"  Is Time Diff < Bar Length? {time_diff < self.bar_length}")

                # Exit loop if the last bar is recent enough (within one bar_length of current time)
                if time_diff < self.bar_length:
                    print("--- Historical data up-to-date enough. Exiting get_most_recent loop. ---") # DEBUG
                    break # Exit the while loop

                else:
                    print("--- Last bar is older than bar_length. Looping again to get more recent data. ---")


            except v20.errors.ResponseNoField as e:
                 # Catch the specific error from the traceback
                 print(f"ERROR during get_history: {e}")
                 print("      This often indicates invalid request parameters (e.g., dates).")
                 print("      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***")
                 traceback.print_exc() # Print full traceback
                 print("Waiting before retry...")
                 time.sleep(10) # Wait longer after API errors

            except Exception as e:
                print(f"UNEXPECTED ERROR during get_history or processing: {e}") # DEBUG
                traceback.print_exc() # Print full traceback for debugging
                print("Waiting before retry...")
                time.sleep(5)

        print("--- Exiting get_most_recent ---") # DEBUG

    def start_trading(self, days): # NEW
        try:
            self.get_most_recent(days)
            # Only start streaming if get_most_recent was successful (didn't raise SystemExit)
            print("\n--- Starting Data Stream ---")
            self.stream_data(self.instrument) # This will block until stopped or error
            print("\n--- Data Stream Stopped ---")
        except SystemExit as e:
             print(f"Trading initialization failed: {e}")
        except Exception as e:
             print(f"An error occurred during trading: {e}")
             traceback.print_exc()


    def on_success(self, time_str, bid, ask): # time is likely a string from stream
        # Add a counter print inside the callback if needed
        # print(f"Tick: {self.ticks} | Time: {time_str} | Bid: {bid} | Ask: {ask}", end='\r', flush=True)

        if self.stop_stream: # Check if termination was requested
             print("Stop stream flag is set, skipping tick processing.")
             return

        # --- Convert incoming time string to NAIVE UTC Timestamp ---
        try:
            # OANDA streams provide time in RFC3339 format (UTC)
            # pd.to_datetime usually handles this well and makes it tz-aware (UTC)
            aware_tick_time = pd.to_datetime(time_str)
            # Convert to naive UTC to match self.last_bar for comparisons
            recent_tick = aware_tick_time.tz_localize(None) # Make naive
        except Exception as e:
             print(f"\nError converting stream timestamp '{time_str}': {e}")
             # Decide how to handle: skip tick, use last known good time? For now, skip.
             return

        # Define stop - Check this *before* expensive operations if possible
        # Adjust the number of ticks as needed for testing
        if self.ticks >= 500: # Reduced for quicker testing if desired
            print("\nDEBUG: Reached tick limit.")
            if not self.stop_stream: # Ensure terminate_session is called only once
                 self.terminate_session(cause = f"Scheduled Session End after {self.ticks} ticks.")
            return # Exit callback

        # --- Check if last_bar is initialized ---
        # This check prevents the TypeError if get_most_recent failed silently
        if self.last_bar is None:
            print("\nERROR: self.last_bar is None. Cannot process ticks. Historical data fetch likely failed.")
            # Consider stopping the stream here if this state persists
            if not self.stop_stream:
                self.terminate_session(cause="Initialization Error: last_bar not set.")
            return

        # Collect and store tick data (using naive UTC index)
        df = pd.DataFrame({self.instrument:(ask + bid)/2},
                          index = [recent_tick])
        # Use pd.concat - consider efficiency for very high frequency
        self.tick_data = pd.concat([self.tick_data, df])

        # --- Check if a new bar needs to be formed ---
        # Compare naive recent_tick with naive self.last_bar
        try:
             time_since_last_bar = recent_tick - self.last_bar
             # print(f"Tick time: {recent_tick}, Last Bar: {self.last_bar}, Diff: {time_since_last_bar}, BarLength: {self.bar_length}") # Fine-grained Debug
             if time_since_last_bar >= self.bar_length:
                 # DEBUG: Log when resampling is triggered
                 print(f"\nDEBUG: Triggering resample. Recent Tick: {recent_tick}, Last Bar: {self.last_bar}, Diff: {time_since_last_bar}")
                 self.resample_and_join()
                 self.define_strategy()
                 self.execute_trades()
        except TypeError as e:
             # This could happen if somehow last_bar becomes None *after* the initial check
             print(f"\nERROR comparing timestamps: {e}. recent_tick={recent_tick} ({type(recent_tick)}), self.last_bar={self.last_bar} ({type(self.last_bar)})")
             if not self.stop_stream:
                self.terminate_session(cause="Timestamp comparison error during stream.")
             return
        # REMOVED Duplicate check block that was here


    def resample_and_join(self):
        print("DEBUG: Entering resample_and_join")
        if self.tick_data.empty:
             print("DEBUG: No tick data to resample.")
             return

        # Resample tick_data (which has a naive UTC index)
        # label='right' aligns the timestamp. last() gets the last tick in the interval.
        # Using iloc[:-1] drops the most recent, potentially incomplete, bar formed from ticks.
        # Using ffill() before dropping seems unusual - normally you'd resample, maybe drop, then maybe fill.
        # Let's stick to the original logic for now, but be aware.
        resampled_ticks = self.tick_data.resample(self.bar_length,
                                                  label="right").last().ffill().iloc[:-1] # Original logic
        # Alternative: resampled_ticks = self.tick_data.resample(self.bar_length, label="right").last().iloc[:-1] # Drop potentially incomplete last bar


        print(f"DEBUG: Resampled ticks shape: {resampled_ticks.shape}")
        print(f"DEBUG: Resampled ticks index TZ: {resampled_ticks.index.tz}") # Should be None (naive)

        if resampled_ticks.empty:
             print("DEBUG: No full bars formed from tick data resampling.")
             # Update last_bar from the *original* tick data if no new bars formed?
             # Or just keep the old last_bar. Let's keep old one for now.
             # We still need to clear the processed ticks.
             # Find the timestamp of the *last* tick that was considered in the (empty) resample
             # This is tricky. A simpler approach might be to just clear ticks up to the last *potential* bar end.
             # Clear ticks that are older than the start of the *next* potential bar
             # potential_next_bar_start = self.last_bar + self.bar_length
             # self.tick_data = self.tick_data[self.tick_data.index >= potential_next_bar_start]
             # Simpler for now: just clear all ticks if no new bar, although this might lose edge ticks.
             # A safer but complex way: determine the timestamp of the last tick used by resample, clear before that.
             # Let's reset based on the last successful bar for simplicity, expecting ticks will catch up.
             self.tick_data = self.tick_data[self.tick_data.index > self.last_bar]
             return # No new bars to add to raw_data

        # --- Ensure raw_data index is NAIVE UTC before concatenating ---
        # self.raw_data's index *should* be aware UTC from get_history initially.
        if self.raw_data.index.tz is not None:
            print("DEBUG: Converting raw_data index to naive UTC before concat.")
            self.raw_data.index = self.raw_data.index.tz_localize(None)
        elif self.raw_data.index.tz is None:
             # If it's already naive (e.g., after a previous resample_and_join), do nothing.
             # print("DEBUG: raw_data index is already naive.")
             pass


        # Concatenate historical (now naive) data with new bars from ticks (naive)
        self.raw_data = pd.concat([self.raw_data, resampled_ticks])

        # Ensure no duplicate indices if resampling overlaps - keep the newest data
        self.raw_data = self.raw_data[~self.raw_data.index.duplicated(keep='last')]
        print(f"DEBUG: raw_data shape after concat: {self.raw_data.shape}")


        # --- Update last_bar to the latest timestamp in raw_data (must be NAIVE UTC) ---
        if not self.raw_data.empty:
            new_last_bar = self.raw_data.index[-1]
            # Ensure it's naive (it should be after the concat logic above)
            if hasattr(new_last_bar, 'tzinfo') and new_last_bar.tzinfo is not None:
                 print(f"WARNING: new_last_bar was aware ({new_last_bar.tzinfo}) after concat, converting.")
                 self.last_bar = new_last_bar.tz_localize(None) # Convert to NAIVE UTC
            else:
                 self.last_bar = new_last_bar # Already naive

            print(f"DEBUG: Updated self.last_bar (Naive UTC): {self.last_bar}")

            # Clean up tick_data: Keep only ticks that are strictly newer than the new last bar
            # This prevents reprocessing ticks that formed the bar we just added.
            self.tick_data = self.tick_data[self.tick_data.index > self.last_bar]
            print(f"DEBUG: Remaining tick_data size after cleanup: {self.tick_data.shape[0]}")
        else:
             # This case should ideally not happen if checks above work
             print("WARNING: raw_data became empty during resample_and_join.")
             # What should last_bar be? Maybe revert to the last tick time? Risky.
             # Keep the old self.last_bar? Or set to None and handle? Let's keep old one.
             print(f"DEBUG: Keeping previous self.last_bar: {self.last_bar}")
             # Clear tick data based on the old bar
             self.tick_data = self.tick_data[self.tick_data.index > self.last_bar]


    def define_strategy(self): # "strategy-specific"
        # Make a copy to avoid modifying raw_data unintentionally
        # The index of self.raw_data should be naive UTC at this point
        df = self.raw_data.copy()
        print(f"DEBUG: Defining strategy on data with index TZ: {df.index.tz}") # Should be None

        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        # Ensure window is > 0 for rolling calculation
        if self.window > 0 :
             df["sma"] = df[self.instrument].rolling(self.window).mean() # Optional: For context
             df["position"] = -np.sign(df.returns.rolling(self.window).mean())
             # Handle potential NaN from rolling mean in the position column
             df['position'] = df['position'].ffill().fillna(0) # Forward fill, then fill remaining NaN with 0 (neutral)
        else:
             # Handle cases where window might be 0 or negative (e.g., default to neutral)
             print("WARN: Window is <= 0, cannot calculate rolling mean. Setting position to 0.")
             df["sma"] = df[self.instrument] # Or NaN, or skip
             df["position"] = 0
        #***********************************************************************

        self.data = df.copy()
        # print(f"DEBUG: Strategy data tail:\n{self.data.tail()}") # Optional debug

    def execute_trades(self):
        # Ensure data and position column exist and have values
        if self.data is None or 'position' not in self.data.columns or self.data.empty:
             print("WARN: Strategy data not ready, cannot execute trades.")
             return

        # Get the latest signal
        signal = self.data["position"].iloc[-1]

        print(f"DEBUG: Executing trades. Current Position: {self.position}, Target Signal: {signal}")

        # Target position is LONG
        if signal == 1:
            if self.position == 0:
                print("Trade: Going Long")
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
                self.position = 1 # Update position based on successful order (?) - create_order doesn't confirm fill
            elif self.position == -1:
                print("Trade: Closing Short, Going Long")
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG (closing short)")
                self.position = 1
            # else: position is already 1, do nothing

        # Target position is SHORT
        elif signal == -1:
            if self.position == 0:
                print("Trade: Going Short")
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
                self.position = -1
            elif self.position == 1:
                print("Trade: Closing Long, Going Short")
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT (closing long)")
                self.position = -1
            # else: position is already -1, do nothing

        # Target position is NEUTRAL
        elif signal == 0:
            if self.position == -1:
                print("Trade: Closing Short, Going Neutral")
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL (closing short)")
                self.position = 0
            elif self.position == 1:
                print("Trade: Closing Long, Going Neutral")
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL (closing long)")
                self.position = 0
            # else: position is already 0, do nothing

    def report_trade(self, order, going):
        # Check if order is valid (tpqoa might return None or an error dict on failure)
        if order is None or not isinstance(order, dict) or "time" not in order:
             print("\n" + 100* "-")
             print(f"TRADE REPORT: Order failed or invalid response for action '{going}'. Order: {order}")
             print(100 * "-" + "\n")
             # Should not update self.position if order failed.
             # The execute_trades logic currently assumes success. More robust logic
             # would confirm the order fill status before updating self.position.
             return

        time = order.get("time", "N/A") # Use .get for safety
        units = order.get("units", "N/A")
        price = order.get("price", "N/A")
        pl = float(order.get("pl", 0.0)) # Use .get and ensure float conversion
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print(f"{time} | {going}")
        # Use f-string for cleaner formatting
        print(f"{time} | units = {units} | price = {price} | P&L = {pl:.2f} | Cum P&L = {cumpl:.2f}")
        print(100 * "-" + "\n")

    def terminate_session(self, cause): # NEW
        print(f"\nTerminating Session: {cause}")
        if not hasattr(self, 'stop_stream') or self.stop_stream:
             print("Termination already in progress or stream not started.")
             return # Avoid duplicate actions

        self.stop_stream = True # Signal to stop processing ticks in on_success

        # Close any open position
        if self.position != 0:
            print(f"Closing position: {self.position * self.units} units of {self.instrument}")
            close_units = -self.position * self.units
            try:
                # Adding a timeout might be wise for cleanup orders
                 close_order = self.create_order(self.instrument, units=close_units,
                                                suppress=True, ret=True) # Consider timeout=10
                 self.report_trade(close_order, f"CLOSING FINAL POSITION ({'Long' if self.position == 1 else 'Short'})")
                 self.position = 0 # Assume close order works for termination
            except Exception as e:
                 print(f"ERROR closing final position: {e}")
                 # Log this error, manual intervention might be needed
        else:
            print("No open position to close.")

        # The stream might stop itself once stop_stream is true and the callback returns,
        # or you might need a more explicit stop depending on how tpqoa handles blocking.
        # If stream_data blocks indefinitely, you might need threading/asyncio
        # or modify tpqoa's stream logic if possible.
        # For now, setting stop_stream is the main mechanism provided by this class structure.
        print("Termination request processed.")


# --- Main Execution ---
if __name__ == "__main__":
    # Ensure your oanda.cfg file is correct and has valid credentials/account ID
    try:
        trader = ConTrader("oanda.cfg", "EUR_USD", bar_length="1min", window = 1, units = 100)

        # *** BEFORE RUNNING: CHECK YOUR SYSTEM CLOCK! ***

        trader.start_trading(days = 5) # Start the process

    except FileNotFoundError:
        print("ERROR: oanda.cfg not found. Please ensure the configuration file is in the correct directory.")
    except Exception as e:
        print(f"An error occurred during initialization or execution: {e}")
        traceback.print_exc()


--- Entering get_most_recent ---

Loop Iteration: 1
Fetching history from 2025-04-07 02:46:53 UTC to 2025-04-12 02:46:53 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A46%3A53.527907%2B00%3A00.000000000Z&to=2025-04-07T08%3A46%3A53.527907%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_26804\3262540947.py", line 53, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A46%3A53.5279


Loop Iteration: 2
Fetching history from 2025-04-07 02:47:05 UTC to 2025-04-12 02:47:05 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A47%3A05.979775%2B00%3A00.000000000Z&to=2025-04-07T08%3A47%3A05.979775%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_26804\3262540947.py", line 53, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A47%3A05.9797


Loop Iteration: 3
Fetching history from 2025-04-07 02:47:18 UTC to 2025-04-12 02:47:18 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A47%3A18.110718%2B00%3A00.000000000Z&to=2025-04-07T08%3A47%3A18.110718%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


...


Loop Iteration: 9
Fetching history from 2025-04-07 02:48:31 UTC to 2025-04-12 02:48:31 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A48%3A31.057916%2B00%3A00.000000000Z&to=2025-04-07T08%3A48%3A31.057916%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_26804\3262540947.py", line 53, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A48%3A31.0579


Loop Iteration: 10
Fetching history from 2025-04-07 02:48:43 UTC to 2025-04-12 02:48:43 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A48%3A43.188303%2B00%3A00.000000000Z&to=2025-04-07T08%3A48%3A43.188303%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_26804\3262540947.py", line 53, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A48%3A43.1883


Loop Iteration: 11
DEBUG: Exiting loop after 10 iterations. Check API errors or clock.
Trading initialization failed: Failed to get recent data after 10 attempts. Check system clock and OANDA connection.


In [8]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [9]:
# This execution code needs to replaced.

# trader.get_most_recent()
# trader.stream_data(trader.instrument, stop = 100)
# if trader.position != 0:
#     close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units,
#                                       suppress = True, ret = True) 
#     trader.report_trade(close_order, "GOING NEUTRAL")
#     trader.position = 0

In [10]:
trader.start_trading(days = 5)


--- Entering get_most_recent ---

Loop Iteration: 1
Fetching history from 2025-04-07 02:48:55 UTC to 2025-04-12 02:48:55 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A48%3A55.327450%2B00%3A00.000000000Z&to=2025-04-07T08%3A48%3A55.327450%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_26804\3262540947.py", line 53, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A48%3A55.3274


Loop Iteration: 2
Fetching history from 2025-04-07 02:49:07 UTC to 2025-04-12 02:49:07 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A49%3A07.711207%2B00%3A00.000000000Z&to=2025-04-07T08%3A49%3A07.711207%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_26804\3262540947.py", line 53, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A49%3A07.7112


Loop Iteration: 3
Fetching history from 2025-04-07 02:49:19 UTC to 2025-04-12 02:49:19 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A49%3A19.840794%2B00%3A00.000000000Z&to=2025-04-07T08%3A49%3A19.840794%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


...


Loop Iteration: 9
Fetching history from 2025-04-07 02:50:32 UTC to 2025-04-12 02:50:32 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A50%3A32.621101%2B00%3A00.000000000Z&to=2025-04-07T08%3A50%3A32.621101%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_26804\3262540947.py", line 53, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A50%3A32.6211


Loop Iteration: 10
Fetching history from 2025-04-07 02:50:44 UTC to 2025-04-12 02:50:44 UTC
ERROR during get_history: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A50%3A44.739764%2B00%3A00.000000000Z&to=2025-04-07T08%3A50%3A44.739764%2B00%3A00.000000000Z does not have field 'candles' (contains 'errorMessage')
      This often indicates invalid request parameters (e.g., dates).
      *** PLEASE VERIFY YOUR SYSTEM CLOCK IS CORRECT! ***
Waiting before retry...


Traceback (most recent call last):
  File "C:\Users\audri\AppData\Local\Temp\ipykernel_26804\3262540947.py", line 53, in get_most_recent
    df = self.get_history(instrument = self.instrument, start = past_aware, end = now_aware,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 232, in get_history
    batch = self.retrieve_data(instrument, batch_start, batch_end,
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\tpqoa\tpqoa.py", line 167, in retrieve_data
    raw = raw.get('candles')
          ^^^^^^^^^^^^^^^^^^
  File "C:\Users\audri\anaconda3\Lib\site-packages\v20\response.py", line 35, in get
    raise ResponseNoField(self, field)
v20.errors.ResponseNoField: 400 response for GET https://api-fxpractice.oanda.com:443/v3/instruments/EUR_USD/candles?price=M&granularity=S5&from=2025-04-07T02%3A50%3A44.7397


Loop Iteration: 11
DEBUG: Exiting loop after 10 iterations. Check API errors or clock.
Trading initialization failed: Failed to get recent data after 10 attempts. Check system clock and OANDA connection.


### Added Wrapper methodes for start and terminate Trading Sessions